In [101]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
import math

In [4]:
#电商特征
df_ecommerce=pd.read_csv('/Users/apple/Documents/天骊宝盒综合报告_1.csv',encoding='gbk')

In [5]:
#运营商特征读取
df_basic=pd.read_csv('/Users/apple/Documents/all_result/all_operator_feat/df_basic.txt')
df_bill=pd.read_csv('/Users/apple/Documents/all_result/all_operator_feat/df_bill.txt')
df_call=pd.read_csv('/Users/apple/Documents/all_result/all_operator_feat/df_call.txt')
df_package=pd.read_csv('/Users/apple/Documents/all_result/all_operator_feat/df_package.txt')
df_recharge=pd.read_csv('/Users/apple/Documents/all_result/all_operator_feat/df_recharge.txt')
df_sms=pd.read_csv('/Users/apple/Documents/all_result/all_operator_feat/df_sms.txt')

In [6]:
#运营商特征合并
df1=df_basic.merge(df_bill,on='mobile',how='inner')
df2=df1.merge(df_call,on='mobile',how='inner')
df3=df2.merge(df_package,on='mobile',how='inner')
df4=df3.merge(df_recharge,on='mobile',how='inner')
df5=df4.merge(df_sms,on='mobile',how='inner')

In [8]:
df_operator=df5

In [12]:
#获取样本标签
df_sample_all=pd.read_csv('/Users/apple/Documents/samples/tianli_all_sample2.txt')

In [34]:
#合并特征宽表
df_sample_all['mobile']=df_sample_all['phone']
df1=df_sample_all.merge(df_ecommerce,on='mobile',how='inner')
df_feature_label=df1.merge(df_operator,on='mobile',how='inner')

In [37]:
df_feature_label.shape

(7601, 3799)

In [26]:
df_feature_label['start_time']=pd.to_datetime(df_feature_label['start_time'])

In [28]:
#特征样本按照申请时间降序排
df_feature_label=df_feature_label.sort_values(['start_time'])

In [39]:
#验证集,取样本10%
valid=df_feature_label.tail(1000)

In [50]:
data=df_feature_label.head(df_feature_label.shape[0]-1000)

In [76]:
#去掉特征文件无关字段
drop_columns=['phone_x',
 'start_time',
 'label',
 'name_x',
 'idcard_x',
 'mobile',
 'name_y',
 'idcard_y',
 'input_timestamp',
'zm_score_update_time',
'ecommerce_update_time','idcard_related_other_names', 
'idcard_related_other_phones', 
'phone_related_other_names', 
'phone_related_other_idcards', 
'name_idcard_matched_alipay', 
'receive_phone_is_same', 
'name_on_alipay', 
'idcard_on_alipay', 
'mobile_on_alipay', 
'carrier_base_availablebalance', 
'carrier_base_level']

In [77]:
feature = list(data.columns)
for name in drop_columns:
    feature.remove(name)

In [84]:
#训练集和测试集划分
x_train, x_test, y_train, y_test = train_test_split(data[feature], data["label"], train_size=0.7, random_state=101)

In [88]:
dtrain = xgb.DMatrix(x_train[feature], label=y_train)
dtest = xgb.DMatrix(x_test[feature], label=y_test)

In [91]:
#参数设置
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist = [(dtest, 'eval'), (dtrain, 'train')]

In [92]:
#训练模型
num_round = 10
bst = xgb.train(param, dtrain, num_round, evallist)

[0]	eval-auc:0.634779	train-auc:0.663732
[1]	eval-auc:0.649896	train-auc:0.695031
[2]	eval-auc:0.66304	train-auc:0.716337
[3]	eval-auc:0.670888	train-auc:0.733073
[4]	eval-auc:0.676109	train-auc:0.744477
[5]	eval-auc:0.680726	train-auc:0.752994
[6]	eval-auc:0.684337	train-auc:0.759094
[7]	eval-auc:0.684792	train-auc:0.765021
[8]	eval-auc:0.687093	train-auc:0.770289
[9]	eval-auc:0.679844	train-auc:0.775264


In [95]:
#预测
ypred = bst.predict(dtest)

In [103]:
#分值转换
for i in range(ypred.size):
    test_score=int(math.log2((1-ypred[i])/ypred[i])*50+450)
    print(test_score)

451
559
471
364
447
557
582
526
417
599
369
510
293
335
357
467
377
393
397
446
505
253
585
445
482
455
573
310
407
355
456
462
524
496
388
539
471
404
401
360
526
537
327
115
245
477
389
419
301
303
450
533
543
452
476
468
389
431
299
452
400
349
359
409
675
410
439
476
366
293
437
493
459
452
454
529
344
469
584
501
425
468
333
418
370
366
355
552
421
371
560
528
508
274
493
334
386
508
413
370
369
448
369
526
394
395
411
533
451
282
444
363
355
376
420
480
527
309
533
507
281
517
312
446
338
379
421
584
311
443
434
449
346
454
496
541
452
416
392
447
331
564
482
476
308
389
618
453
389
451
428
372
400
447
560
304
370
390
351
301
380
320
410
538
372
438
630
443
581
239
493
281
339
486
404
466
362
354
249
526
503
547
416
443
397
548
360
393
436
276
513
446
395
542
462
203
372
494
443
377
378
505
520
445
299
525
469
310
449
482
357
554
482
523
460
445
563
506
495
534
466
511
434
582
433
446
356
527
530
417
513
422
320
488
398
565
430
452
337
529
425
305
430
569
408
506
289
401
403
467
